### Author: Bogdan Bintu and Benjamin Alberts
     Date: 10/6/2022
    This is inteded to create genome-scale probes with ~250kb resolution across the human genome

In [2]:
!cd LibraryDesign3/C_Tools && python setup.py build_ext --inplace

running build_ext
building 'seqint' extension
creating build/temp.linux-x86_64-3.9
gcc -pthread -B /usr/local/anaconda/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /usr/local/anaconda/include -I/usr/local/anaconda/include -fPIC -O2 -isystem /usr/local/anaconda/include -fPIC -I/usr/local/anaconda/lib/python3.9/site-packages/numpy/core/include -I/usr/local/anaconda/include/python3.9 -c ./seqint.c -o build/temp.linux-x86_64-3.9/./seqint.o
In file included from /usr/local/anaconda/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1969,
                 from /usr/local/anaconda/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/anaconda/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4,
                 from ./seqint.c:700:
/usr/local/anaconda/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using depreca

In [3]:
import sys,os
sys.path.append(r'LibraryDesign3')
import LibraryDesigner as ld
import LibraryTools as lt
from seqint import seq2Int, seq2Int_rc
# some other packages may required
import Bio #use pip install biopython

In [4]:
#### Downloaded T2T genome from: https://www.ncbi.nlm.nih.gov/assembly/GCA_009914755.4
#### GCF_009914755.1_T2T-CHM13v2.0_genomic.fna

In [5]:
names,seqs = lt.fastaread(r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna')

FileNotFoundError: [Errno 2] No such file or directory: 'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'

### Construct count tables -- only need to do it once

In [ ]:
fl = r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace('.fna','_17w.bin'),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

100%|███████████████████████████████████████████| 24/24 [08:32<00:00, 21.36s/it]


Time to compute unique and clip: 430.0407826900482
Time to update matrix: 4.8050079345703125


In [ ]:
import numpy as np
arr = np.fromfile(r'HumanGenomeTranscriptome/human_genome_T2T_17w.bin',dtype=np.uint16)

seq = b"ccctaaccctaacccta"
arr[ld.seq2Int(seq)] #number of occurences in the genome of ccctaaccctaacccta

In [ ]:
fl = r'HumanTranscriptome/human__genesUnspliced.fasta'
names,seqs = lt.fastaread(fl)
ct = ld.countTable(word=17,save_file=fl.replace('.fasta','_17w.bin'),
                   sparse=False)

from tqdm import tqdm
for sq in tqdm(seqs):
    ct.consume(sq,verbose=False)
ct.complete(verbose=True)
ct.save()

### sample genome at 250kb and save 50kb centered

TO DO

Save these as fata files index by chr and location: chr5_locus_0025_50kb.fasta

In [ ]:
fl = r'HumanGenome/GCF_009914755.1_T2T-CHM13v2.0_genomic.fna'
names_gn,seqs_gn = lt.fastaread(fl)

In [ ]:
def save_fasta_loci(name_,seq_,resolution=50000,spacing=250000,save_folder = r'Probes'):
    """
    Given <name_> and <seq_> of the chromosome and resolution and spacing, 
    tile the chromome with <spacing> and save each segment of size <resolution> with a name as a fasta file.
    """
    chr_ = name_.split('chromosome')[-1].split(',')[0].replace(' ','')
    len_ = len(seq_)
    nloci = int(len_/spacing)
    for ilocus in tqdm(np.arange(nloci)):
        start = spacing*ilocus
        seq = seq_[start:(start+resolution)]#[:resolution]
        res = int(resolution/1000)
        name = 'chr'+chr_+'_locus'+str(ilocus).zfill(5)+'_'+str(res)+'kb'
        fl_name = save_folder+os.sep+name+'.fasta'
        lt.fastawrite(fl_name,[name],[seq])

In [ ]:
for i in range(len(names_gn)):
    name_,seq_ = names_gn[i],seqs_gn[i]
    save_fasta_loci(name_,seq_,resolution=50000,spacing=250000,save_folder = r'Probes')

### Run on single sequence the probe designing

In [ ]:
names[4]
save_folder = r'Probes'
ich=4
index = 25

name = 'chr5_locus'+str(index).zfill(5)+'_50kb'
seq = seqs[ich][index*250000:][:50000]
fl_name = save_folder+os.sep+name+'.fasta'
lt.fastawrite(fl_name,[name],[seq])

In [ ]:
print(fl_name)

In [2]:
in_file = r'Probes/chr5_locus00025_50kb.fasta'

save_file = in_file.replace('.fasta','.pbr')

transcriptome_fl = r'HumanTranscriptome/human__genesUnspliced_17w.bin'
genome_fl = r'HumanGenome/human_genome_T2T_17w.bin'
rep_fl = r'HumanGenome/repetitive.fasta'
top_transcriptome_fl = r'HumanTranscriptome/top_tr500_brain.fasta'
local_genome_fl = in_file


pb_designer = ld.pb_reports_class(
    sequence_dic={'file':in_file,'use_revc':True,'use_kmer':True}, #use_revc - if true  consides rc as probe
    
    map_dic={'transcriptome':{'file':transcriptome_fl,'use_revc':False,'use_kmer':True},
             'top_transcriptome':{'file':top_transcriptome_fl,'use_revc':False,'use_kmer':True},
          'genome':{'file':genome_fl,'use_revc':True,'use_kmer':True},
          'repetitive':{'file':rep_fl,'use_revc':True,'use_kmer':True},
          'local_genome':{'file':in_file,'use_revc':True,'use_kmer':True}},

    save_file=save_file,
    
    params_dic={'word_size':17,'pb_len':40,'buffer_len':0,'max_count':2**16-1,'check_on_go':True,'auto':False},
    
    dic_check={('genome','local_genome'):75,
                'transcriptome':20,
                'top_transcriptome':5,
                'repetitive':2,
               'gc':[0.25,0.75],'tm':70})

pb_designer.computeOTmaps()
pb_designer.compute_pb_report(verbose=True)
pb_designer.perform_check_end()
pb_designer.plots()
pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

Setting attribute: map_transcriptome
Setting attribute: map_top_transcriptome
Mapping no. of seqs: 3689
Setting attribute: map_genome
Setting attribute: map_repetitive
Mapping no. of seqs: 67029
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Time(s): 563.7516188621521
Dealing with sequence: 1 out of 1


100%|███████████████████████████████████| 49960/49960 [00:07<00:00, 6685.67it/s]


Time(s): 74.88342595100403
Picking non-overlaping sequences.
Time(s): 58.822932720184326


In [ ]:
#### Define in_files

In [7]:
import glob
import numpy as np
in_files = glob.glob('Probes/chr*.fasta')

In [9]:
import random
random.shuffle(in_files)

In [10]:
from tqdm.notebook import tqdm
for in_file in tqdm(in_files):
    #print(in_file)
    #modify input file
    in_file=str(in_file)
    pb_designer.sequence_dic['file'] = in_file
    #modify save file
    pb_designer.save_file = in_file.replace('.fasta','.pbr')
    if not os.path.exists(pb_designer.save_file):
        pb_designer.load_sequence_file_and_paramaters()
        #modify maps
        key='local_genome'
        pb_designer.map_dic[key]['file'] = in_file
        pb_designer.files_to_OTmap("map_"+key,pb_designer.map_dic[key])
        #compute
        pb_designer.compute_pb_report()
        pb_designer.perform_check_end()
        pb_designer.plots()
        pb_designer.save_csv(name=os.path.basename(in_file).split('.')[0])

  0%|          | 0/12460 [00:00<?, ?it/s]

Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 10.430143117904663
Picking non-overlaping sequences.
Time(s): 1.955284595489502
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 9.127468347549438
Picking non-overlaping sequences.
Time(s): 18.863338708877563
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 78.06186747550964
Picking non-overlaping sequences.
Time(s): 2.3226304054260254
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 21.595008850097656
Picking non-overlaping sequences.
Time(s): 2.379739284515381
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 10.803467512130737
Picking non-overlaping sequences.
Time(s): 2.0843899250030518
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out o

Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 17.373695373535156
Picking non-overlaping sequences.
Time(s): 1.9098966121673584
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 49.04516625404358
Picking non-overlaping sequences.
Time(s): 2.3068313598632812
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 53.4257538318634
Picking non-overlaping sequences.
Time(s): 1.9941787719726562
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 47.35773015022278
Picking non-overlaping sequences.
Time(s): 2.4693524837493896
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 64.07339477539062
Picking non-overlaping sequences.
Time(s): 2.6017298698425293
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 7.923952341079712
Pickin

Dealing with sequence: 1 out of 1
Time(s): 17.826098680496216
Picking non-overlaping sequences.
Time(s): 1.644411325454712
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 15.3454749584198
Picking non-overlaping sequences.
Time(s): 2.2626259326934814
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 9.758231163024902
Picking non-overlaping sequences.
Time(s): 2.6098592281341553
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 8.974165916442871
Picking non-overlaping sequences.
Time(s): 2.201922655105591
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 69.17433333396912
Picking non-overlaping sequences.
Time(s): 2.3234591484069824
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 7.85221529006958
Picking non-overlaping sequences

Time(s): 9.28469181060791
Picking non-overlaping sequences.
Time(s): 2.2954390048980713
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 69.50328135490417
Picking non-overlaping sequences.
Time(s): 2.7940785884857178
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 21.324975728988647
Picking non-overlaping sequences.
Time(s): 3.255117177963257
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 74.38222813606262
Picking non-overlaping sequences.
Time(s): 2.871410369873047
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 15.462005138397217
Picking non-overlaping sequences.
Time(s): 2.5254006385803223
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 12.92721676826477
Picking non-overlaping sequences.
Time(s): 2.368997812271118
Set

...

Dealing with sequence: 1 out of 1
Time(s): 17.54649567604065
Picking non-overlaping sequences.
Time(s): 2.552537441253662
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 6.092569351196289
Picking non-overlaping sequences.
Time(s): 2.0474472045898438
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 68.60969614982605
Picking non-overlaping sequences.
Time(s): 2.6995365619659424
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 9.776094675064087
Picking non-overlaping sequences.
Time(s): 2.4184155464172363
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 17.36143207550049
Picking non-overlaping sequences.
Time(s): 2.61816668510437
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 68.24289727210999
Picking non-overlaping sequences

Time(s): 2.5274500846862793
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 86.29800581932068
Picking non-overlaping sequences.
Time(s): 2.4265458583831787
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 84.75433826446533
Picking non-overlaping sequences.
Time(s): 2.667407751083374
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 78.05020666122437
Picking non-overlaping sequences.
Time(s): 2.6625587940216064
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 82.36647772789001
Picking non-overlaping sequences.
Time(s): 3.4196314811706543
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 65.6058886051178
Picking non-overlaping sequences.
Time(s): 2.217583179473877
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealin

Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 87.01819014549255
Picking non-overlaping sequences.
Time(s): 3.4342710971832275
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 85.8550488948822
Picking non-overlaping sequences.
Time(s): 3.3067898750305176
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 72.63774538040161
Picking non-overlaping sequences.
Time(s): 2.0002472400665283
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 85.50791358947754
Picking non-overlaping sequences.
Time(s): 2.664825677871704
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 89.88964986801147
Picking non-overlaping sequences.
Time(s): 1.9810235500335693
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1

Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 104.38084721565247
Picking non-overlaping sequences.
Time(s): 2.667997121810913
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 109.55574011802673
Picking non-overlaping sequences.
Time(s): 2.681330442428589
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 97.69249176979065
Picking non-overlaping sequences.
Time(s): 1.9021801948547363
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 98.53346252441406
Picking non-overlaping sequences.
Time(s): 2.291799306869507
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 105.70578455924988
Picking non-overlaping sequences.
Time(s): 8.087620496749878
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 93.84046196937561
Picking

Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 102.62858438491821
Picking non-overlaping sequences.
Time(s): 2.5611367225646973
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 102.25541996955872
Picking non-overlaping sequences.
Time(s): 2.4057085514068604
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 118.66792678833008
Picking non-overlaping sequences.
Time(s): 2.5881335735321045
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 105.55768013000488
Picking non-overlaping sequences.
Time(s): 2.205580711364746
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 109.12548995018005
Picking non-overlaping sequences.
Time(s): 2.5346617698669434
Setting attribute: map_local_genome
Mapping no. of seqs: 1
Dealing with sequence: 1 out of 1
Time(s): 111.01002550125122
P

In [124]:
names__,seqs__ = lt.fastaread(in_file)
map_ = ld.OTmap(seqs__,word_size=17,use_kmer=True,progress_report=False,save_file=None,sparse=False)

Mapping no. of seqs: 1


In [122]:
??ld.OTmap

In [116]:
ct = ld.countTable(word=17,save_file=None,sparse=False)
ct.consume(pb_designer.input_seqs[0],verbose=False)

In [119]:
cints,ctsints = np.unique(ct.ints,return_counts=True)
len(cints)

48708

In [39]:
np.sum([int(len(seq)/250000) for seq in seqs])

12460

In [ ]:
384 well plate - 3000$

In [40]:
12460/384*3000

97343.75

8577